# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

import mlflow
import mlflow.sklearn

import nbimporter
import fonctions_from_notebook_1

# Initialisation MLFlow

In [ ]:
#mlflow.set_tracking_uri("http://localhost:5000")  # Définir l'URI de suivi de MLflow (facultatif)
#mlflow.set_experiment("OC - Projet 6 - Traitement image")

: 

# Préparation des datas pour l'entrainement

In [2]:
images_dir_OG= "./Datas/Images"
annotations_dir_OG= "./Datas/Annotation"
images_dir_S= "./Datas/Images_Small"
annotations_dir_S= "./Datas/Annotation_Small"
output_size=(224, 224)

In [3]:
from tensorflow.keras.utils import img_to_array
datas = fonctions_from_notebook_1.get_dogs_picture_breed(images_dir_OG,annotations_dir_OG,output_size)

Processing directories:  10%|▉         | 12/121 [00:13<02:18,  1.27s/it, current_directory=./Datas/Images\n02088364-beagle]             

In [ ]:
labels = datas['breed'].unique()
label_to_index = {label: idx for idx, label in enumerate(labels)}
datas['breed_idx'] = datas['breed'].map(label_to_index)

In [ ]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
from PIL import Image
import numpy as np

target_shape = (224, 224)
processed_images = []

for img in datas['image']:
    # Check if the image is a numpy array or a PIL Image
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)  # Convert numpy array to PIL Image
    
    # Now img is guaranteed to be a PIL Image
    if img.size != target_shape:
        print("Y")
        img = resize_image(img, target_shape)
    img = np.array(img)  # Convert PIL Image back to numpy array
    processed_images.append(img)

ImportError: cannot import name 'img_to_array' from 'keras.preprocessing.image' (e:\Users\Beast-Of-War\miniconda3\envs\tf2_10\lib\site-packages\keras\preprocessing\image.py)

In [ ]:
# Séparer les données en images et étiquettes
X = np.array(img_to_array(datas['image']))
y = to_categorical(datas['breed_idx'])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(labels), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Geler les couches de VGG16
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

e:\Users\Beast-Of-War\miniconda3\envs\tf2_10\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# Entraîner le modèle
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stopping]
)

# Évaluation sur les données de validation
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {val_accuracy*100:.2f}%')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Image).

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convertir les probabilités en classes
y_true = np.argmax(y_val, axis=1)  # Convertir les étiquettes one-hot en classes

report = classification_report(y_true, y_pred_classes, target_names=labels)
print(report)

: 